## How to run this service

* Launch a terminal and enter into the directory, that contains this notebook
* then type
```
preview-nb-api.sh API-worksheet.ipynb 
```
* the first line of output will tell you the URL of the API

In [ ]:
import json
import base64
import pandas as pd

In [63]:
fname = 'Globalco2emissions.csv'
#fname = "Global CO2 emissions - CDIAC and UN Population.csv"
df = pd.read_csv(fname, parse_dates=[1])
df['Year'] = pd.to_datetime( df['Year'])
#df['Year'] = df['Year'].dt.strftime('%Y')

In [ ]:
REQUEST = json.dumps({
 'args' : {}
})

In [ ]:
# GET /help
help_txt = ("""
<h2>HELP</h2>
<h4>GET /info</h4>
Description: Retrieve either some informatin about the dataset
Requires: No parameters required
Example query: /api/info
Example result: {"name": "Global CO2 emissions - CDIAC and UN Population", "title": "Global CO2 emissions .... }

<h4>GET /data</h4>
Description: Retrieves either the all the data, or the from and until the given dates
Requires: 
<ul>
<li> from (int)</li>
<li> until (int)</li>
</ul>
Returns: 
<ul>
<li> year</li>
<li> metric tons per capita </li>
</ul>
Example query: /api/data?from=1965&until=1988

<h4>POST /data/</h4>
Description: Adds an extra data point into the dataset for the given 'year' with the given 'value'
Requires: 
<ul>
<li> year (int) </li>
<li> value (float)</li>
</ul>
Returns: 
<ul>
<li> Status 200: upload successful</li>
</ul>
Example query: /api/data?year=2016&value=3.9152

<h4>PUT /data/</h4>
Description: Replaces anexisting data point in the dataset for the given 'year' with the given 'value'
Requires: 
<ul>
<li> year (int) </li>
<li> value (float)</li>
</ul>
Returns: 
<ul>
<li> Status 200: upload successful</li>
</ul>
Example query: /api/data?year=2016&value=3.9152

<h4>GET /agg</h4>
<ul>
<li>Description: Retrieves an aggregated sum of the CO2 consumption for the last <years>. </li>
<li>Requires: years </li>
<li>Returns: a number</li>
<li>Example query: /api/agg?interval=6</li>
</ul>
""")
help_txt=help_txt.replace('\n',"")

print(json.dumps(help_txt))

In [ ]:
# ResponseInfo GET /help
print(json.dumps({
    "headers" : {
        "Content-Type" : "text/html"
    },
    "status" : 200
}))


In [ ]:
# GET /api/info
req = json.loads(REQUEST)
args = req['args']

resp = {
    "name":"Global CO2 emissions - CDIAC and UN Population",
    "title":"Global CO2 emissions - CDIAC and UN Population",
    "dataset_id": 652,
    "description":"Per capita CO2 emissions have been calculated based on the combination of global emissions data from CDIAC (described below), and UN Population Prospects data.\r\n\r\nPopulation data was derived from: The History Database of the Global Environment (HYDE) collected the data by earlier publications. For the 'OurWorldInData'-series we used various sources: The data for the period before 1900 are taken from the History Database of the Global Environment (HYDE). The data for the World Population between 1900 and 1940 is taken from the UN puplication 'The World at Six Billion'. The annual data for the World Population between 1950 and 2015 is taken from the UN's World Population Prospects: The 2015 Revision. It is the series 'Total Population - Both Sexes' online available at: https://esa.un.org/unpd/wpp/.\r\n\r\nEmissions data have been sourced from the Carbon Dioxide Information Analysis Centre (CDIAC) database. Emissions data have been converted from units of carbon to carbon dioxide (CO2) using a conversion factor of 3.67.\r\n\r\nCDIAC denote a \"statistical difference\" component which has been included in this data. This statistical difference represents the difference between estimated global CO2 emissions and the sum of national totals. Estimates of CO2 emissions show that the global total of emissions is not equal to the sum of emissions from all countries. This is introduced in several cases: emissions within international territories, which are included in global totals but not attributed to individual countries; inconsistent national reporting where global import and export data is imbalanced; and differing treatment of non-fuel uses of hydrocarbons.\r\n\r\nFull methodology on global, regional, national and statistical difference estimations can be found in Le Quere et al. (2016): Le Quéré, C., Andrew, R. M., Canadell, J. G., Sitch, S., Korsbakken, J. I., Peters, G. P., ... & Keeling, R. F. (2016). Global carbon budget 2016. Earth System Science Data, 8(2), 605. Available at: doi:10.5194/essd-8-605-2016.",
    "sources": {
      "name": "Global CO2 emissions - CDIAC and UN Population",
      "dataPublishedBy": "Carbon Dioxide Information Analysis Center (CDIAC)",
      "dataPublisherSource": "Boden, T. A. and Andres, R. J.: Global, Regional, and National Fossil-Fuel CO2 Emissions. Oak Ridge National Laboratory, US Department of Energy, Oak Ridge, Tenn., USA (2016)",
      "link": "http://cdiac.ornl.gov/CO2_Emission/",
      "retrievedDate": "31/07/2017"
    }
}


print(json.dumps(resp))

In [ ]:
# ResponseInfo GET /api/info
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

In [ ]:
# GET /api/data
req = json.loads(REQUEST)
args = req['args']

if not "from" in args or not "until" in args:
    response = {"Error": "Missing argument. Expecting both 'from' and 'until'"}
    print (json.dumps(response))
else:
    date_from = pd.to_datetime(args["from"][0])
    date_until = pd.to_datetime(args["until"][0])
    if date_from >= date_until:
        response = {"queryParams" : query_params_dict, "message" : "'from' should be an earlier year than until"}
        print (json.dumps (response))
    elif (date_from > df.Year.max()) or (date_until<df.Year.min()):
        query_params_dict = {"from" : args["from"][0], "until" : args["until"][0]}
        response = {
            "queryParams" : query_params_dict,
            "message" : "valid range for 'from' and 'until' is between {} and {} ".format(df['Year'].dt.strftime('%Y').min(), 
                                                                                          df['Year'].dt.strftime('%Y').max())
        }
        print (json.dumps (response))
    else:
        rows = df[(df["Year"] >= date_from) &
                             (df["Year"] <= date_until)][["Year", "Per capita CO2 emissions (CDIAC and UN Population)"]].copy()
        data_dict = [{"date" : date.strftime('%Y'), "metric tons per capita" : numbers} for date, numbers in rows.values ]
        query_params_dict = {"from" : args["from"][0], "until" : args["until"][0]}
        response = {
            "queryParams" : query_params_dict, 
            "data" : data_dict
        }
        print (json.dumps (response))

        
# curl -X GET "https://kooplex-edu.elte.hu/notebook/wfct0p-rere-wfct0p/report/api/data?from=1718&until=1919"

In [ ]:
# ResponseInfo GET /api/data
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

In [ ]:
## Insert new data with POST

In [ ]:
# PSTOST /api/data
req = json.loads(REQUEST)
args = req['args']

if not "date" in args or not "value" in args:
    response = { "Error" : "Missing argument. Expecting both 'date' and 'value'" }
    print (json.dumps (response))
else:
    date = args["date"][0]
    value = float(args["value"][0])
    if (df.Year.dt.strftime("%Y")==date).sum():
        response = {
            "Error" : "'date' should not be an existing value"
            }
        print (json.dumps (response))
    else:
        new_row = {'Year': pd.to_datetime(date),
                   'CO2 emissions (CDIAC and UN Population)': value,
                   'Per capita CO2 emissions (CDIAC and UN Population)': value,
                   'Entity': 'World'}
        df = df.append(new_row, ignore_index=True)
        df.to_csv(fname, index=False, index_label=None)
        response = {
            "Message" : "upload successful"
        }
        print (json.dumps (response))


# curl -X POST "https://kooplex-edu.elte.hu/notebook/wfct0p-rere-wfct0p/report/api/data?date=1718&value=2.324"

In [ ]:
# ResponseInfo POST /api/data
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

In [ ]:
## Replace existing data with PUT

In [ ]:
# PUT /api/data
req = json.loads(REQUEST)
args = req['args']

if not "date" in args or not "value" in args:
    response = {
    "Error" : "Missing argument. Expecting both 'date' and 'value'"
    }
    print (json.dumps (response))
else:
    date = args["date"][0]
    value = float(args["value"][0])
    if not (df.Year.dt.strftime("%Y")==date).sum():
        response = {
            "Error" : "'date' should be an existing one"
            }
        print (json.dumps (response))
    else:
        df.loc[df.Year.dt.strftime("%Y")==date, 'Per capita CO2 emissions (CDIAC and UN Population)'] = value
        df.to_csv(fname, index=False, index_label=None)
        response = {
            "Message" : "Data has been changed",
            "Year": date,
            "New value": value
        }
        print (json.dumps (response))

In [ ]:
# GET /api/agg
req = json.loads(REQUEST)
args = req['args']

response = {
    "Error" : "This function is not implemented yet" 
}
print(json.dumps (response))

In [ ]:
# # GET /api/image
# req = json.loads(REQUEST)
# args = req['args']

# import base64

# with open("images/outerhebrydes.jpeg",'rb') as i:
#     img = i.read()
#     img_encoded = base64.encodebytes(img).decode()

# print(json.dumps(img_encoded))

In [ ]:
# # ResponseInfo GET /api/image
# print(json.dumps({
#     "headers" : {
#         "Content-Type" : "image/jpeg"
#     },
#     "status" : 200
# }))

In [ ]:
# GET /:nothingfound

print ("You seem lost, try /help")

In [ ]:
# GET /

print ("You seem lost, try /help")